In [5]:
from fastapi import FastAPI, HTTPException
import secrets
import time

app = FastAPI()

# 토큰 저장소 (DB 대신 메모리 사용, 실제 배포 시 DB 사용 추천)
TOKENS = {}
address = 'https://chatgpt.com/g/g-67a089879d0c819184d793d8ab8ed247-ai-essay'

# ✅ **토큰 생성 API** (관리자가 실행)
def generate_auth_link(user_email: str):
    token = secrets.token_urlsafe(32)  # 랜덤한 토큰 생성
    TOKENS[token] = {"email": user_email, "expires": time.time() + 86400}  # 24시간 유효
    return f"{address}/auth?token={token}"

In [6]:
# ✅ **이메일 수동 발송**
# 구매 확인 후, 아래 명령어를 실행하여 링크를 이메일에 포함하여 보내세요

email = "user@example.com"
auth_link = generate_auth_link(email)
print(f"Send this link to the user: {auth_link}")


Send this link to the user: https://chatgpt.com/g/g-67a089879d0c819184d793d8ab8ed247-ai-essay/auth?token=Y91JXf3Mcno2lAd3anEhd7CdAYU2qxk0aFBnm2BxFrM


In [7]:


# ✅ **사용자가 링크 클릭하면 자동 로그인**
@app.get("/auth")
def authenticate(token: str):
    if token not in TOKENS or time.time() > TOKENS[token]["expires"]:
        raise HTTPException(status_code=401, detail="Invalid or expired token")

    user_email = TOKENS[token]["email"]
    return {"message": "Authentication successful!", "email": user_email, "access_granted": True}


In [8]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import RedirectResponse
import time

app = FastAPI()

# ✅ 토큰 저장소 (DB 대신 임시 사용)
TOKENS = {
    "abc": {
        "expires": time.time() + 86400,  # 24시간 유효
        "redirect_url": "https://chatgpt.com/g/g-67a089879d0c819184d793d8ab8ed247-ai-essay"
    }
}

@app.get("/auth")
def authenticate(token: str):
    # ✅ 토큰 유효성 검사
    if token not in TOKENS or time.time() > TOKENS[token]["expires"]:
        raise HTTPException(status_code=401, detail="Invalid or expired token")

    # ✅ ChatGPT 맞춤형 GPT 페이지로 리디렉트
    chatgpt_url = TOKENS[token]["redirect_url"]
    return RedirectResponse(url=chatgpt_url)

